Sequence-to-Sequence模型  
<center><img src="seq2seq.jpg" alt="FAO" width="500"></center> 
1.可用于机器翻译  
2.文章摘要  
3.对话机器人  
4.中文分词  
......

In [1]:
# -*- coding:utf-8 -*-
import re
import numpy as np
import pandas as pd

# 
text = open('msr_train_10.txt').read()
text = text.split('\n')

In [2]:
len(text)

10

In [3]:
# {B:begin, M:middle, E:end, S:single}，分别代表每个状态代表的是该字在词语中的位置，
# B代表该字是词语中的起始字，M代表是词语中的中间字，E代表是词语中的结束字，S则代表是单字成词
text

['“/s  人/b  们/e  常/s  说/s  生/b  活/e  是/s  一/s  部/s  教/b  科/m  书/e  ，/s  而/s  血/s  与/s  火/s  的/s  战/b  争/e  更/s  是/s  不/b  可/m  多/m  得/e  的/s  教/b  科/m  书/e  ，/s  她/s  确/b  实/e  是/s  名/b  副/m  其/m  实/e  的/s  ‘/s  我/s  的/s  大/b  学/e  ’/s  。/s  ',
 '“/s  心/s  静/s  渐/s  知/s  春/s  似/s  海/s  ，/s  花/s  深/s  每/s  觉/s  影/s  生/s  香/s  。/s  ',
 '“/s  吃/s  屎/s  的/s  东/b  西/e  ，/s  连/s  一/s  捆/s  麦/s  也/s  铡/s  不/s  动/s  呀/s  ？/s  ',
 '他/s  “/s  严/b  格/m  要/m  求/e  自/b  己/e  ，/s  从/s  一/b  个/e  科/b  举/e  出/b  身/e  的/s  进/b  士/e  成/b  为/e  一/b  个/e  伟/b  大/e  的/s  民/b  主/m  主/m  义/e  者/s  ，/s  进/b  而/e  成/b  为/e  一/s  位/s  杰/b  出/e  的/s  党/b  外/e  共/b  产/m  主/m  义/e  战/b  士/e  ，/s  献/b  身/e  于/s  崇/b  高/e  的/s  共/b  产/m  主/m  义/e  事/b  业/e  。/s  ',
 '“/s  征/s  而/s  未/s  用/s  的/s  耕/b  地/e  和/s  有/s  收/b  益/e  的/s  土/b  地/e  ，/s  不/b  准/e  荒/b  芜/e  。/s  ',
 '“/s  这/s  首/b  先/e  是/s  个/s  民/b  族/e  问/b  题/e  ，/s  民/b  族/e  的/s  感/b  情/e  问/b  题/e  。/s  ',
 '’/s  我/s  扔/s  了/s  两/b  颗/e  手/b  榴/m  弹/e

In [4]:
# 设置参数
# 词向量长度
word_size = 128
# 设置最长的一句话为32个字
maxlen = 32
# 批次大小
batch_size = 1024

# 根据符号分句
text = u''.join( text)
text = re.split(u'[，。！？、]/[bems]', text)

In [5]:
len(text)

25

In [6]:
text

['“/s  人/b  们/e  常/s  说/s  生/b  活/e  是/s  一/s  部/s  教/b  科/m  书/e  ',
 '  而/s  血/s  与/s  火/s  的/s  战/b  争/e  更/s  是/s  不/b  可/m  多/m  得/e  的/s  教/b  科/m  书/e  ',
 '  她/s  确/b  实/e  是/s  名/b  副/m  其/m  实/e  的/s  ‘/s  我/s  的/s  大/b  学/e  ’/s  ',
 '  “/s  心/s  静/s  渐/s  知/s  春/s  似/s  海/s  ',
 '  花/s  深/s  每/s  觉/s  影/s  生/s  香/s  ',
 '  “/s  吃/s  屎/s  的/s  东/b  西/e  ',
 '  连/s  一/s  捆/s  麦/s  也/s  铡/s  不/s  动/s  呀/s  ',
 '  他/s  “/s  严/b  格/m  要/m  求/e  自/b  己/e  ',
 '  从/s  一/b  个/e  科/b  举/e  出/b  身/e  的/s  进/b  士/e  成/b  为/e  一/b  个/e  伟/b  大/e  的/s  民/b  主/m  主/m  义/e  者/s  ',
 '  进/b  而/e  成/b  为/e  一/s  位/s  杰/b  出/e  的/s  党/b  外/e  共/b  产/m  主/m  义/e  战/b  士/e  ',
 '  献/b  身/e  于/s  崇/b  高/e  的/s  共/b  产/m  主/m  义/e  事/b  业/e  ',
 '  “/s  征/s  而/s  未/s  用/s  的/s  耕/b  地/e  和/s  有/s  收/b  益/e  的/s  土/b  地/e  ',
 '  不/b  准/e  荒/b  芜/e  ',
 '  “/s  这/s  首/b  先/e  是/s  个/s  民/b  族/e  问/b  题/e  ',
 '  民/b  族/e  的/s  感/b  情/e  问/b  题/e  ',
 '  ’/s  我/s  扔/s  了/s  两/b  颗/e  手/b  榴/m  弹/e

In [7]:
# 训练集数据
data = []
# 标签
label = []

# 得到所有的数据和标签
def get_data(s):
    s = re.findall('(.)/(.)', s)
    if s:
        s = np.array(s)
        # 返回数据和标签，0为数据，1为标签
        return list(s[:,0]), list(s[:,1])

for s in text:
    d = get_data(s)
    if d:
        data.append(d[0])
        label.append(d[1])

In [8]:
test = re.findall('(.)/(.)', '你/s  只/b  有/e  把/s  事/b  情/e  做/b  好/e')
test

[('你', 's'),
 ('只', 'b'),
 ('有', 'e'),
 ('把', 's'),
 ('事', 'b'),
 ('情', 'e'),
 ('做', 'b'),
 ('好', 'e')]

In [9]:
# 定义一个dataframe存放数据和标签
d = pd.DataFrame(index=range(len(data)))
d['data'] = data
d['label'] = label
# 提取data长度小于等于maxlen的数据
d = d[d['data'].apply(len) <= maxlen]
# 重新排列index
d.index = range(len(d))

In [10]:
d

,data,label
0,"[“, 人, 们, 常, 说, 生, 活, 是, 一, 部, 教, 科, 书]","[s, b, e, s, s, b, e, s, s, s, b, m, e]"
1,"[而, 血, 与, 火, 的, 战, 争, 更, 是, 不, 可, 多, 得, 的, 教, ...","[s, s, s, s, s, b, e, s, s, b, m, m, e, s, b, ..."
2,"[她, 确, 实, 是, 名, 副, 其, 实, 的, ‘, 我, 的, 大, 学, ’]","[s, b, e, s, b, m, m, e, s, s, s, s, b, e, s]"
3,"[“, 心, 静, 渐, 知, 春, 似, 海]","[s, s, s, s, s, s, s, s]"
4,"[花, 深, 每, 觉, 影, 生, 香]","[s, s, s, s, s, s, s]"
5,"[“, 吃, 屎, 的, 东, 西]","[s, s, s, s, b, e]"
6,"[连, 一, 捆, 麦, 也, 铡, 不, 动, 呀]","[s, s, s, s, s, s, s, s, s]"
7,"[他, “, 严, 格, 要, 求, 自, 己]","[s, s, b, m, m, e, b, e]"
8,"[从, 一, 个, 科, 举, 出, 身, 的, 进, 士, 成, 为, 一, 个, 伟, ...","[s, b, e, b, e, b, e, s, b, e, b, e, b, e, b, ..."
9,"[进, 而, 成, 为, 一, 位, 杰, 出, 的, 党, 外, 共, 产, 主, 义, ...","[b, e, b, e, s, s, b, e, s, b, e, b, m, m, e, ..."


In [11]:
#统计所有字，给每个字编号
chars = [] 
for i in data:
    chars.extend(i)

chars = pd.Series(chars).value_counts()

In [12]:
chars

的    14
“     9
一     6
主     5
是     5
有     5
义     4
不     4
个     4
产     3
我     3
而     3
民     3
出     3
大     3
共     3
科     3
战     2
书     2
跟     2
为     2
进     2
族     2
接     2
你     2
问     2
’     2
没     2
也     2
题     2
     ..
才     1
要     1
具     1
两     1
活     1
连     1
溜     1
名     1
扔     1
动     1
原     1
业     1
部     1
火     1
把     1
伟     1
海     1
捆     1
觉     1
求     1
常     1
和     1
只     1
影     1
系     1
己     1
与     1
在     1
静     1
外     1
dtype: int64

In [13]:
chars[:] = range(1, len(chars)+1)
chars

的      1
“      2
一      3
主      4
是      5
有      6
义      7
不      8
个      9
产     10
我     11
而     12
民     13
出     14
大     15
共     16
科     17
战     18
书     19
跟     20
为     21
进     22
族     23
接     24
你     25
问     26
’     27
没     28
也     29
题     30
    ... 
才    132
要    133
具    134
两    135
活    136
连    137
溜    138
名    139
扔    140
动    141
原    142
业    143
部    144
火    145
把    146
伟    147
海    148
捆    149
觉    150
求    151
常    152
和    153
只    154
影    155
系    156
己    157
与    158
在    159
静    160
外    161
dtype: int32

In [14]:
#生成适合模型输入的格式
from keras.utils import np_utils

# 定义标签所对应的编号
tag = pd.Series({'s':0, 'b':1, 'm':2, 'e':3, 'x':4})

# # 把中文变成编号，再补0
# d['x'] = d['data'].apply(lambda x: np.array(list(chars[x])+[0]*(maxlen-len(x))))
# # 把标签变成编号，再补0
# d['y'] = d['label'].apply(lambda x: np.array(list(map(lambda y:np_utils.to_categorical(y,5), tag[x].reshape((-1,1))))+[np.array([[0,0,0,0,1]])]*(maxlen-len(x))))


def data_helper(x):
    x = list(chars[x]) + [0]*(maxlen-len(x))
    return np.array(x)     

def label_helper(x):
    x = list(map(lambda y:np_utils.to_categorical(y,5), tag[x].reshape((-1,1))))
    x = x + [np.array([[0,0,0,0,1]])]*(maxlen-len(x))
    return np.array(x) 
    
d['x'] = d['data'].apply(data_helper) 
d['y'] = d['label'].apply(label_helper)    

Using TensorFlow backend.


In [15]:
d['data'][0]

['“', '人', '们', '常', '说', '生', '活', '是', '一', '部', '教', '科', '书']

In [16]:
d['x'][0]

array([  2, 130,  50, 152,  98,  51, 136,   5,   3, 144,  34,  17,  19,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0])

In [17]:
d['label'][0]

['s', 'b', 'e', 's', 's', 'b', 'e', 's', 's', 's', 'b', 'm', 'e']

In [18]:
d['y'][0]

array([[[1., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0.]],

       [[0., 0., 0., 1., 0.]],

       [[1., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0.]],

       [[0., 0., 0., 1., 0.]],

       [[1., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0.]],

       [[0., 0., 1., 0., 0.]],

       [[0., 0., 0., 1., 0.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 1.]],

       [

<center><img src="lstm1.png" alt="FAO" width="500"></center> 
<center><img src="lstm2.png" alt="FAO" width="500"></center> 

In [19]:
# 设计模型
from keras.layers import Dense, Embedding, LSTM, TimeDistributed, Input, Bidirectional
from keras.models import Model
from keras.models import load_model

sequence = Input(shape=(maxlen,), dtype='int32')
# 词汇数，词向量长度，输入的序列长度，是否忽略0值
embedded = Embedding(len(chars)+1, word_size, input_length=maxlen, mask_zero=True)(sequence)
# 双向RNN包装器
blstm = Bidirectional(LSTM(64, return_sequences=True), merge_mode='sum')(embedded)
# 该包装器可以把一个层应用到输入的每一个时间步上
output = TimeDistributed(Dense(5, activation='softmax'))(blstm)
# 定义模型输出输出
model = Model(inputs=sequence, outputs=output)
# 定义代价函数，优化器
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# print(np.array(list(d['x'])).shape)
# print(np.array(list(d['y'])).reshape((-1,maxlen,5)).shape)
# model.fit(np.array(list(d['x'])), np.array(list(d['y'])).reshape((-1,maxlen,5)), batch_size=batch_size, epochs=20)
# model.save('seq2seq.h5')

print("load model")
model = load_model('seq2seq.h5')

load model


In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 32, 128)           660864    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 64)            98816     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 5)             325       
Total params: 760,005
Trainable params: 760,005
Non-trainable params: 0
_________________________________________________________________


做预测

In [21]:
# 使用全数据
text = open('msr_train.txt').read()
text = text.split('\n')

# 根据符号分句
text = u''.join(text)
text = re.split(u'[，。！？、]/[bems]', text)

# 训练集数据
data = []
# 标签
label = []

# 得到所有的数据和标签
def get_data(s):
    s = re.findall('(.)/(.)', s)
    if s:
        s = np.array(s)
        # 返回数据和标签，0为数据，1为标签
        return list(s[:,0]), list(s[:,1])

for s in text:
    d = get_data(s)
    if d:
        data.append(d[0])
        label.append(d[1])
        
# 定义一个dataframe存放数据和标签
d = pd.DataFrame(index=range(len(data)))
d['data'] = data
d['label'] = label
# 提取data长度小于等于maxlen的数据
d = d[d['data'].apply(len) <= maxlen]
# 重新排列index
d.index = range(len(d))

#统计所有字，给每个字编号
chars = [] 
for i in data:
    chars.extend(i)

chars = pd.Series(chars).value_counts()
chars[:] = range(1, len(chars)+1)

In [22]:
# 统计状态转移
dict_label = {}
for label in d['label']:
    for i in range(len(label)-1):
        tag = label[i] + label[i+1]
        dict_label[tag] = dict_label.get(tag,0) + 1
print(dict_label)

{'eb': 528062, 'be': 949884, 'ss': 266154, 'sb': 412351, 'es': 378288, 'mm': 170711, 'bm': 183969, 'me': 183974}


In [23]:
# 计算状态转移总次数
sum_num = 0
for value in dict_label.values():
    sum_num = sum_num + value
sum_num

3073393

In [24]:
# 计算状态转移概率
p_ss = dict_label['ss']/sum_num
p_sb = dict_label['sb']/sum_num
p_bm = dict_label['bm']/sum_num
p_be = dict_label['be']/sum_num
p_mm = dict_label['mm']/sum_num
p_me = dict_label['me']/sum_num
p_es = dict_label['es']/sum_num
p_eb = dict_label['eb']/sum_num

In [25]:
# 维特比算法，维特比算法是一种动态规划算法用于寻找最有可能产生观测事件序列的-维特比路径

# tag = pd.Series({'s':0, 'b':1, 'm':2, 'e':3, 'x':4})

# 00 = ss = 1
# 01 = sb = 1
# 02 = sm = 0
# 03 = se = 0
# 10 = bs = 0
# 11 = bb = 0
# 12 = bm = 1
# 13 = be = 1
# 20 = ms = 0
# 21 = mb = 0
# 22 = mm = 1
# 23 = me = 1
# 30 = es = 1
# 31 = eb = 1
# 32 = em = 0
# 33 = ee = 0

# 定义状态转移矩阵
transfer = [[p_ss,p_sb,0,0],
            [0,0,p_bm,p_be],
            [0,0,p_mm,p_me],
            [p_es,p_eb,0,0]]

# # 定义状态转移矩阵
# transfer = [[1,1,0,0],
#             [0,0,1,1],
#             [0,0,1,1],
#             [1,1,0,0]]

# 根据符号断句
cuts = re.compile(u'([\da-zA-Z ]+)|[。，、？！\.\?,!]')

# 预测分词
def predict(sentence):
    
    # 如果句子大于最大长度，只取maxlen个词
    if len(sentence) > maxlen:
        sentence = sentence[:maxlen]
    
    # 预测结果，先把句子编程编号的形式，如果出现生僻字就填充0，然后给句子补0直到maxlen的长度。预测得到的结果只保留跟句子有效数据相同的长度
    result = model.predict(np.array([list(chars[list(sentence)].fillna(0).astype(int))+[0]*(maxlen-len(sentence))]))[0][:len(sentence)]

    # 存放最终结果
    y = []
    # 存放临时概率值
    prob = []
    # 计算最大转移概率
    # 首先计算第1个词和第2个词,统计16种情况的概率
    # result[0][j]第1个词的标签概率
    # result[1][k]第2个词的标签概率
    # transfer[j][k]对应的转移概率矩阵的概率
    for j in range(4):
        for k in range(4):
            prob.append(result[0][j]*result[1][k]*transfer[j][k])
    # 计算前一个词的的标签
    word1 = np.argmax(prob)//4
    # 计算后一个词的标签
    word2 = np.argmax(prob)%4
    # 保存结果
    y.append(word1)
    y.append(word2)
    # 从第2个词开始
    for i in range(1,len(sentence)-1):
        # 存放临时概率值
        prob = []
        # 计算前一个词后后一个词的所有转移概率
        for j in range(4):
            prob.append(result[i][word2]*result[i+1][j]*transfer[word2][j])
        # 计算后一个词的标签
        word2 = np.argmax(prob)%4
        # 保存结果
        y.append(word2)
        
    # 分词
    words = []
    for i in range(len(sentence)):
        # 如果标签为s或b，append到结果的list中
        if y[i] in [0, 1]:
            words.append(sentence[i])
        else:
        # 如果标签为m或e，在list最后一个元素中追加内容
            words[-1] += sentence[i]
    return words

# 分词
def cut_word(s):
    result = []
    # 指针设置为0
    j = 0
    # 根据符号断句
    for i in cuts.finditer(s):
        # 对符号前的部分分词
        result.extend(predict(s[j:i.start()]))
        # 加入符号
        result.append(s[i.start():i.end()])
        # 移动指针到符号后面
        j = i.end()
    # 对最后的部分进行分词
    result.extend(predict(s[j:]))
    return result

In [26]:
cut_word('基于seq2seq的中文分词器')

['基于', 'seq2seq', '的', '中文', '分', '词器']

In [27]:
cut_word('人们常说生活是一部教科书')

['人们', '常说', '生活', '是', '一', '部', '教科书']

In [28]:
cut_word('广义相对论是描写物质间引力相互作用的理论')

['广义', '相对论', '是', '描写', '物', '质', '间', '引力', '相互', '作用', '的', '理论']

In [29]:
model.predict(np.array([list(chars[list('今天天气很好')].fillna(0).astype(int))+[0]*(maxlen-len('今天天气很好'))]))[0]

array([[5.3695333e-04, 9.9946302e-01, 2.5521592e-08, 4.3570618e-09,
        1.4844584e-09],
       [1.9184376e-04, 7.1968236e-07, 1.0082910e-06, 9.9980646e-01,
        2.5343561e-09],
       [1.1647578e-06, 9.9999511e-01, 3.7472551e-06, 4.4786855e-11,
        1.2289153e-11],
       [4.9465605e-05, 1.5763385e-10, 6.9671858e-04, 9.9925381e-01,
        4.1510535e-11],
       [9.9982244e-01, 1.7732578e-04, 1.8819671e-07, 1.0828031e-08,
        5.8034572e-10],
       [9.9905795e-01, 4.6581710e-07, 8.5055262e-08, 9.4140600e-04,
        1.6101223e-08],
       [1.4285019e-01, 2.0312425e-05, 9.3301540e-05, 8.5702705e-01,
        9.1915881e-06],
       [1.4285019e-01, 2.0312425e-05, 9.3301540e-05, 8.5702705e-01,
        9.1915881e-06],
       [1.4285019e-01, 2.0312425e-05, 9.3301540e-05, 8.5702705e-01,
        9.1915881e-06],
       [1.4285019e-01, 2.0312425e-05, 9.3301540e-05, 8.5702705e-01,
        9.1915881e-06],
       [1.4285019e-01, 2.0312425e-05, 9.3301540e-05, 8.5702705e-01,
        9.19